# 📈: Text Augmentation using large-scale LMs and prompt engineering

This notebook looks into the possibility of performing data augmentation on an NLP dataset leveraging the few-shot capabilities of large-scale language models (LMs) and prompt engineering 💡.

Data augmentation techniques are used to generate additional samples. Data augmentation is already standard practice in computer vision projects 👀, but can also be leveraged in many NLP problems. We'll use a limited training set to simulate a real-world use case, where we are often constrained by the size of the available data 🤦.

## 🛠️ Getting started

The cells below will setup everything that is required to get started with data augmentation and finetuning an NLP model with the HuggingFace API.

### Setup

In [ ]:
!pip install -q transformers==4.12.5 datasets==1.16.1 tokenizers==0.10.3 openai==0.11.3 requests==2.23.0 sentencepiece==0.1.96 pandas==1.1.0

### Imports

In [2]:
import re
import os
import json
import torch
import pickle
import random
import requests
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from datasets import load_dataset, concatenate_datasets, load_from_disk, load_metric, Dataset, ClassLabel
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, TrainerCallback, AutoModelForCausalLM, AutoModelForSeq2SeqLM

### Download dataset
We'll train and evaluate our models on [Emotion](https://huggingface.co/datasets/emotion) dataset that contains English Twitter messages labeled as one of the six basic emotions: anger, fear, joy, love, sadness and surprise. To reduce the complexity of the task, we will keep only three labels, namely:
- joy 😂
- anger 😠
- surprise 😯

In [ ]:
# load the dataset and filter on samples that have a token count less than 30 
# to use only short tweets
max_input_len = 30
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
emotion_ds = load_dataset("emotion").filter(lambda e: len(tokenizer.batch_encode_plus([e['text']]).input_ids[0]) < int(max_input_len))

The dataset is already split into 16,000 train and 2,000 test samples. To investigate the effectiveness of the augmentation method, we will use only 10 samples per class as a train set.

In [ ]:
# select 10 random train samples from each of the three emotions
# SADNESS = 0
# JOY = 1
# LOVE = 2
# ANGER = 3
# FEAR = 4
# SURPRISE = 5 

joy_train_samples = emotion_ds['train'].filter(lambda e: e['label'] == 1).shuffle(seed=42).select(range(10))
anger_train_samples = emotion_ds['train'].filter(lambda e: e['label'] == 3).shuffle(seed=42).select(range(10))
surprise_train_samples = emotion_ds['train'].filter(lambda e: e['label'] == 5).shuffle(seed=42).select(range(10))

# map emotions to integers for labeling
# joy (0), anger (1), surprise (2)
def map_emotions(example):
    if example['label'] == 1: # joy
        example['label'] = 0
    elif example['label'] == 3: # anger
        example['label'] = 1
    else: 
        example['label'] = 2 # surprise
    return example

# create a train set that consists of 10 samples per class and filter the test 
# set to contain only the valid labels
emotion_train_ds = concatenate_datasets([joy_train_samples, anger_train_samples, surprise_train_samples]).map(lambda e: map_emotions(e)).shuffle(seed=42)
emotion_test_ds = emotion_ds["test"].filter(lambda e: e['label'] in [1, 3, 5]).map(lambda e: map_emotions(e))

# define the maping between emotions and labels
mapping = ClassLabel(names=['joy', 'anger', 'surprise'])

Before proceeding with the data augmentation, let's have a look into the baseline dataset 😎!

In [7]:
print("Train set")
print(f"Total samples: {len(emotion_train_ds)}\n")
print("A random sample")
print(f"Text: {emotion_train_ds['text'][10]} \nLabel: {mapping.int2str(emotion_train_ds['label'][10])}")
print("\n")

print("Test set")
print(f"Total samples: {len(emotion_test_ds)}\n")
print("A random sample")
print(f"Text: {emotion_test_ds['text'][10]} \nLabel: {mapping.int2str(emotion_test_ds['label'][10])}")

Train set
Total samples: 30

A random sample
Text: i feel angered and firey 
Label: anger


Test set
Total samples: 797

A random sample
Text: i feel more virtuous than when i eat veggies dipped in hummus 
Label: joy


## Text Augmentation pipeline

We will leverage the few-shot capabilities of large LMs to generate synthetic but very realistic samples from a mixture of real samples. Specifically, we select two real samples from our dataset and embed these samples in a carefully designed prompt. Then, the LM takes as input the prompt and generates an augmented mixed sample influenced by the sample sentences.


Generally, a prompt looks like this:

    Each item in the following list contains a <text type> and the
    respective <label type>. <label type> is one of ’<label token 1>’,
    ..., or ’<label token N>’. 
    <text type>: <example text 1> (<label type>: <example label 1>)
    ...
    <text type>: <example text k> (<label type>: <example label k>)
    <text type>:

In our case the prompt looks like this:

    Each item in the following list contains a tweet and the
    respective sentiment. Sentiment is one of ’joy’, 'surprise' or 'anger'. 
    Tweet: i feel angered and firey (Sentiment: anger)
    Tweet: im feeling very peaceful about our wedding again now after having (Sentiment: joy)
    Tweet:

You can find more information on text augmentation using large LMs in [GPT3Mix](https://arxiv.org/abs/2104.08826) paper.

First, we should extract pairs of samples from the train set. There are various extraction strategies that can be used to increase the quality of the synthetic samples. We will simply extract the pairs randomly since by repeating random sampling a diverse synthetic dataset will be created.

In [8]:
def get_two_random_samples():
    # define a function that returns two random samples from the train set.
    s1, s2 = random.sample(range(0, len(emotion_train_ds)), 2)
    return emotion_train_ds['text'][s1], emotion_train_ds['label'][s1], emotion_train_ds['text'][s2], emotion_train_ds['label'][s2]

def get_prompt(text1, label1, text2, label2):
    # define a function that takes as input two samples and generates the prompt
    # that we should pass to the GPT-3 language model for completion.
    description = "Each item in the following list contains a tweet and the respective sentiment. Sentiment is one of 'joy', 'surprise' or 'anger'."
    prompt = (f"{description}\n"
            f"Tweet: {text1} (Sentiment: {mapping.int2str(label1)})\n"
            f"Tweet: {text2} (Sentiment: {mapping.int2str(label2)})\n"
            f"Tweet:")
    return prompt

### GPT-3

We will leverage [GPT-3](https://openai.com/blog/openai-api/) that is a powerful model developed by Open AI and an excellent few-shot learner allowing it to be controlled via natural text prompts. We will generate 10, 50, 100 and 200 synthetic samples using the OpenAI API.

**Disclaimer:** You need a key to make requests to OpenAI API. You can request a key [here](https://beta.openai.com/docs/developer-quickstart) or you just can load the synthetic samples that we generated in the next cell.

In [10]:
# define the number of synthetic samples to generate
n = 10
new_texts = []
new_labels = []
api_key = # insert your api key for GPT-3
headers = {'Authorization' : 'Bearer ' + api_key,
              'Content-type':'application/json', 
              'Accept':'application/json'}

iter = 0
while iter < n:
    # select two random samples from training set
    text1, label1, text2, label2 = get_two_random_samples()
    # create the prompt
    prompt = get_prompt(text1, label1, text2, label2)
    # send a post request to gpt-3 using the prompt
    response = requests.post('https://api.openai.com/v1/engines/davinci/completions',
                             headers=headers,
                             data = json.dumps({"prompt": prompt, 
                                                "max_tokens": 30,
                                                "temperature": 0.9,
                                                "top_p": 0.95}))

    # get response and extract the generated text and label
    # the generated output will be in the form "<text> (Sentiment: <label>)"
    data = response.json()['choices'][0]['text'].split('\n')[0].split('(Sentiment:')

    if len(data) < 2:
        # the format of the response is invalid
        continue

    text = data[0]
    label = data[1].split(')')[0].strip()

    if label not in ['joy', 'anger', 'surprise']:
        # the format of the response is invalid
        continue

    new_texts.append(text)
    new_labels.append(mapping.str2int(label))
    iter += 1

# define the synthetic dataset and save it to disk so as to prevent sending 
# many api requests
synthetic_ds = Dataset.from_dict({'text': new_texts, 'label': new_labels})
synthetic_ds.save_to_disk('./data/gpt-3/' + str(n))

In [4]:
# load the synthetic datasets with 10, 50, 100 and 200 samples
# run this if you do not want to generate samples on your own.

if not os.path.isdir('./data'):
    !git clone https://github.com/ml6team/quick-tips.git
    !cd quick-tips && git checkout feature/nlp_gpt3mix
    !mv quick-tips/nlp/2021_11_25_augmentation_lm/data ./data
    !rm -rf quick-tips

synthetic_gpt3_10_ds = load_from_disk('./data/gpt-3/10')
synthetic_gpt3_50_ds = load_from_disk('./data/gpt-3/50')
synthetic_gpt3_100_ds = load_from_disk('./data/gpt-3/100')
synthetic_gpt3_200_ds = load_from_disk('./data/gpt-3/200')

Now let's print some synthetic samples to examine their quality!

In [5]:
def print_random_sample(ds):
    # function that prints a random sample from a dataset
    idx = random.randint(0, len(ds)-1)
    print(f"Text: {ds['text'][idx]} \nLabel: {mapping.int2str(ds['label'][idx])}\n")

In [8]:
print("Dataset of 10 synthetic samples:")
print_random_sample(synthetic_gpt3_10_ds)
print("Dataset of 50 synthetic samples:")
print_random_sample(synthetic_gpt3_50_ds)
print("Dataset of 100 synthetic samples:")
print_random_sample(synthetic_gpt3_100_ds)
print("Dataset of 200 synthetic samples:")
print_random_sample(synthetic_gpt3_200_ds)

Dataset of 10 synthetic samples:
Text:  even if ur not into these kind of things u have to admit it's pretty cool  
Label: joy

Dataset of 50 synthetic samples:
Text:  i just feel like every time they get off my car they're just ripping me off  
Label: anger

Dataset of 100 synthetic samples:
Text:  i've had this knot in my stomach all week. i thought it was indigestion  
Label: anger

Dataset of 200 synthetic samples:
Text:  20% or more electricity costs due to inefficient appliances and systems. #green #efficiency #Smarthome  
Label: anger



We see that GPT-3 has effectively generated very realistic samples. 👏👏👏

### GPT-Neo

An open-source alternative of GPT-3 is [GPT-Neo](https://www.eleuther.ai/projects/gpt-neo/) by a group called Eleuther AI. The goal of the group is to democratize huge LMs, so they made GPT-Neo publicly available. GPT-3 on the other hand is not openly available at the time. To use GPT-Neo, we can load it through HuggingFace. 

**Disclaimer:** To use GPT-Neo, it would take at least 25GB of CPU RAM to just load the model. So, it might not be able to load it in a regular notebook. In case you want to avoid that, you just can load the synthetic samples that we generated in the next cell.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neo-2.7B')
model = AutoModelForCausalLM.from_pretrained('EleutherAI/gpt-neo-2.7B')

# define the number of synthetic samples to generate
n = 10
new_texts = []
new_labels = []

iter = 0
while iter < n:
    # select two random samples from training set
    text1, label1, text2, label2 = get_two_random_samples()
    # create the prompt
    prompt = get_prompt(text1, label1, text2, label2)

    # generate text using GPT-J model
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    gen_tokens = model.generate(input_ids, do_sample=True, temperature=0.9, max_length=100,)
    gen_text = tokenizer.batch_decode(gen_tokens)[0]
    # the generated output will be in the form "<text> (Sentiment: <label>)"
    data = gen_text.split('\n')[3].strip('Tweet: ').split('(Sentiment:')
    if len(data) < 2:
        # the format of the response is invalid
        continue

    text = data[0]
    label = data[1].split(')')[0].strip()
    if label not in ['joy', 'anger', 'surprise']:
        # the format of the response is invalid
        continue

    new_texts.append(text)
    new_labels.append(mapping.str2int(label))
    iter += 1


# define the synthetic dataset and save it to disk 
synthetic_ds = Dataset.from_dict({'text': new_texts, 'label': new_labels})
synthetic_ds.save_to_disk('./data/gpt-neo/' + str(n))

In [9]:
if not os.path.isdir('./data'):
    !git clone https://github.com/ml6team/quick-tips.git
    !cd quick-tips && git checkout feature/nlp_gpt3mix
    !mv quick-tips/nlp/2021_11_25_augmentation_lm/data ./data
    !rm -rf quick-tips

synthetic_gptneo_10_ds = load_from_disk('./data/gpt-neo/10')
synthetic_gptneo_50_ds = load_from_disk('./data/gpt-neo/50')
synthetic_gptneo_100_ds = load_from_disk('./data/gpt-neo/100')
synthetic_gptneo_200_ds = load_from_disk('./data/gpt-neo/200')

In [ ]:
print("Dataset of 10 synthetic samples:")
print_random_sample(synthetic_gptneo_10_ds)
print("Dataset of 50 synthetic samples:")
print_random_sample(synthetic_gptneo_50_ds)
print("Dataset of 100 synthetic samples:")
print_random_sample(synthetic_gptneo_100_ds)
print("Dataset of 200 synthetic samples:")
print_random_sample(synthetic_gptneo_200_ds)

Dataset of 10 synthetic samples:
Text: happy friday :)  
Label: anger

Dataset of 50 synthetic samples:
Text: his is the easiest time of year to feel unsatisfied  
Label: anger

Dataset of 100 synthetic samples:
Text: he new version of google chat is out  
Label: surprise

Dataset of 200 synthetic samples:
Text: I'm feeling happy and excited  
Label: surprise



GPT-Neo generates realistic samples! However, we can see that there are cases that the label is wrong (example 1).

### GPT-J

A third alternative is [GPT-J](https://6b.eleuther.ai/), which was also made and released by Eleuther AI. Like GPT-Neo, we can access GPT-J through HuggingFace.

**Disclaimer:** To load GPT-J, it would take at least 48GB of CPU RAM to just load the model. So, it might not be able to load it in a regular notebook. In case you want to avoid that, you just can load the synthetic samples that we generated in the next cell.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-j-6B")

# define the number of synthetic samples to generate
n = 10
new_texts = []
new_labels = []

iter = 0
while iter < n:
    # select two random samples from training set
    text1, label1, text2, label2 = get_two_random_samples()
    # create the prompt
    prompt = get_prompt(text1, label1, text2, label2)

    # generate text using GPT-J model
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    gen_tokens = model.generate(input_ids, do_sample=True, temperature=0.9, max_length=100,)
    gen_text = tokenizer.batch_decode(gen_tokens)[0]
    # the generated output will be in the form "<text> (Sentiment: <label>)"
    data = gen_text.split('\n')[3].strip('Tweet: ').split('(Sentiment:')
    if len(data) < 2:
        # the format of the response is invalid
        continue

    text = data[0]
    label = data[1].split(')')[0].strip()
    if label not in ['joy', 'anger', 'surprise']:
        # the format of the response is invalid
        continue

    new_texts.append(text)
    new_labels.append(mapping.str2int(label))
    iter += 1

# define the synthetic dataset and save it to disk 
synthetic_ds = Dataset.from_dict({'text': new_texts, 'label': new_labels})
synthetic_ds.save_to_disk('./data/gpt-j/' + str(n))

In [10]:
# load the synthetic datasets with 10, 50, 100 and 200 samples
# run this if you do not want to generate new samples on your own.

if not os.path.isdir('./data'):
    !git clone https://github.com/ml6team/quick-tips.git
    !cd quick-tips && git checkout feature/nlp_gpt3mix
    !mv quick-tips/nlp/2021_11_25_augmentation_lm/data ./data
    !rm -rf quick-tips

synthetic_gptj_10_ds = load_from_disk('./data/gpt-j/10')
synthetic_gptj_50_ds = load_from_disk('./data/gpt-j/50')
synthetic_gptj_100_ds = load_from_disk('./data/gpt-j/100')
synthetic_gptj_200_ds = load_from_disk('./data/gpt-j/200')

In [ ]:
print("Dataset of 10 synthetic samples:")
print_random_sample(synthetic_gptj_10_ds)
print("Dataset of 50 synthetic samples:")
print_random_sample(synthetic_gptj_50_ds)
print("Dataset of 100 synthetic samples:")
print_random_sample(synthetic_gptj_100_ds)
print("Dataset of 200 synthetic samples:")
print_random_sample(synthetic_gptj_200_ds)

Dataset of 10 synthetic samples:
Text: i think it s the easiest time of year to feel dissatisfied  
Label: anger

Dataset of 50 synthetic samples:
Text: i feel great  
Label: joy

Dataset of 100 synthetic samples:
Text: i'm surprised and have to say "hmmm"  
Label: surprise

Dataset of 200 synthetic samples:
Text: i get giddy over feeling elegant in a beautifully cut pencil skirt  
Label: joy



Even though GPT-J is much smaller than GPT-3, it manages to generate high quality samples 👏.

## 🚀 Model 

Here we define the model and the training pipeline. We will use [DistilBERT](https://arxiv.org/abs/1910.01108) which is a light Transformer trained by distilling BERT base. It has 40% fewer parameters than bert-base-uncased, runs 60% faster while preserving over 95% of BERT’s performances as measured on the GLUE language understanding benchmark.

In [11]:
metric = load_metric("accuracy")

batch_size = 8
max_size = 200 # size of the largest augmented dataset
steps = 5*int(max_size/batch_size) # 5 epochs in the large dataset

run_dicts = [] # list of dicts to store both metrics and logs for all the experiment runs 

# prepare synthetic datasets
datasets_gpt3 = {'10': synthetic_gpt3_10_ds, '50': synthetic_gpt3_50_ds, '100': synthetic_gpt3_100_ds, '200': synthetic_gpt3_200_ds}
datasets_gptneo = {'10': synthetic_gptneo_10_ds, '50': synthetic_gptneo_50_ds, '100': synthetic_gptneo_100_ds, '200': synthetic_gptneo_200_ds}
datasets_gptj = {'10': synthetic_gptj_10_ds, '50': synthetic_gptj_50_ds, '100': synthetic_gptj_100_ds, '200': synthetic_gptj_200_ds}

**Disclaimer:** In case you don't want to train the models again, you can import our results by running only this cell and ignoring the other cells of this section.

In [12]:
# run this if you do not want to train the models on your own.
if not os.path.isdir('./results'):
    !git clone https://github.com/ml6team/quick-tips.git
    !cd quick-tips && git checkout feature/nlp_gpt3mix
    !mv quick-tips/nlp/2021_11_25_augmentation_lm/results ./results
    !rm -rf quick-tips

run_dicts = pickle.load(open('./results/run_dicts.pkl', 'rb'))

In [17]:
def compute_metrics(eval_pred):
    """
        Calculates the accuracy of the model's predictions, calculated as follows; 
                        (TP + TN) / (TP + TN + FP + FN) 
        with TP: True positive TN: True negative FP: False positive FN: False negative
    """
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels) 


class LogAccumulatorCallback(TrainerCallback):
    """
    A class that stores both the training and the evaluation loss
    """
    
    def __init__(self):
        self.acc_logs = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        _ = logs.pop("total_flos", None)
        if state.is_local_process_zero and ('loss' in logs or 'eval_loss' in logs):
            self.acc_logs.append(logs.copy())


def train_and_evaluate(train_ds, test_ds, identifier):
    def tokenize(batch):
        return tokenizer(batch['text'], padding=True, truncation=True)
    
    train_ds = train_ds.map(tokenize, batched=True, batch_size=len(train_ds), remove_columns=["text"])
    test_ds = test_ds.map(tokenize, batched=True, batch_size=len(test_ds), remove_columns=["text"])
    
    training_args = TrainingArguments(
        identifier,
        evaluation_strategy="steps",
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        logging_strategy="steps",
        weight_decay=0.01,
        learning_rate=1e-4,
        max_steps=steps,
        logging_steps=20
    )
    
    model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

    # Partially freezing the weights of initial layers of the model
    # Since we're working on small datasets as it usually reduces overfitting
    # Another advantage of partial freezing is reduced memory usage and 
    # a speed improvement during training.
    for block in model.distilbert.embeddings.modules():
        for param in block.parameters():
            param.requires_grad=False

    for i in [0,1,2]:
        for block in model.distilbert.transformer.layer[i].modules():
            for param in block.parameters():
                param.requires_grad=False

            
    logger = LogAccumulatorCallback()
    trainer = Trainer(
        model=model, args=training_args, 
        train_dataset=train_ds, 
        eval_dataset=test_ds,
        compute_metrics=compute_metrics,
        callbacks=[logger],
    )
    trainer.train()
    metrics = trainer.evaluate()
    
    return metrics, logger.acc_logs

### Model baseline

In [ ]:
# train our model on the baseline dataset without augmentation
metrics, logs = train_and_evaluate(emotion_train_ds, emotion_test_ds, "baseline")

run_dicts.append({
    "id": "baseline",
    "metrics": metrics,
    "logs": logs
})

### Model with augmented data of GPT-3

In [ ]:
# train the model on the augmented datasets of GPT-3
for i in datasets_gpt3:
    augmented_gpt3_ds = concatenate_datasets([emotion_train_ds, datasets_gpt3[i]])
    metrics, logs = train_and_evaluate(augmented_gpt3_ds, emotion_test_ds, "augmented_gpt3_" + i)

    run_dicts.append({
        "id": "augmented_gpt3_" + i,
        "metrics": metrics,
        "logs": logs
    })

### Model with augmented data of GPT-Neo

In [ ]:
# train our model on the augmented datasets of GPT-Neo
for i in datasets_gptneo:
    augmented_gptneo_ds = concatenate_datasets([emotion_train_ds, datasets_gptneo[i]])
    metrics, logs = train_and_evaluate(augmented_gptneo_ds, emotion_test_ds, "augmented_gptneo_" + i)

    run_dicts.append({
        "id": "augmented_gptneo_" + i,
        "metrics": metrics,
        "logs": logs
    })

### Model with augmented data of GPT-J

In [ ]:
# train the model on the augmented datasets of GPT-J
for i in datasets_gptj:
    augmented_gptj_ds = concatenate_datasets([emotion_train_ds, datasets_gptj[i]])
    metrics, logs = train_and_evaluate(augmented_gptj_ds, emotion_test_ds, "augmented_gptj_" + i)

    run_dicts.append({
        "id": "augmented_gptj_" + i,
        "metrics": metrics,
        "logs": logs
    })

##  📊 Visualize

To evaluate the effectiveness of text augmentation in the performance of the model, we'll visualize the results.

### Model Performance

Now let's compare the performance of the trained models. First, we will examine the accuracy improvements of text augmentation using GPT-3, GPT-J and GPT-Neo and then we will compare the three methods.

In [13]:
df = pd.DataFrame(run_dicts)

gpt3_names = ['augmented_gpt3_10', 'augmented_gpt3_50', 'augmented_gpt3_100', 'augmented_gpt3_200']
gptj_names = ['augmented_gptj_10', 'augmented_gptj_50', 'augmented_gptj_100', 'augmented_gptj_200']
gptneo_names = ['augmented_gptneo_10', 'augmented_gptneo_50', 'augmented_gptneo_100', 'augmented_gptneo_200']

# define a dataframe for each LM
df_baseline = df.loc[df['id'] == 'baseline']
df_gpt3 = df.loc[df['id'].isin(gpt3_names)]
df_gptj = df.loc[df['id'].isin(gptj_names)]
df_gptneo = df.loc[df['id'].isin(gptneo_names)]

We observe that the accuracy of the model increases as we augment more and more data. After generating 200 extra synthetic samples, the accuracy exceeds 70% indicating that text augmentation can greatly improve the accuracy of our model.

In [14]:
def plot_accuracy_curve(df, title):
    fig = go.Figure()

    fig.add_trace(go.Scatter(
                        x=list(range(0, steps, 20)),
                        y=pd.DataFrame(df_baseline.loc[0]['logs']).dropna(subset=['eval_accuracy'])['eval_accuracy'],
                        name='accuracy {}'.format('baseline')))

    for index, row in df.iterrows():
        
        fig.add_trace(go.Scatter(
                        x=list(range(0, steps, 20)),
                        y=pd.DataFrame(row['logs']).dropna(subset=['eval_accuracy'])['eval_accuracy'],
                        name='accuracy {}'.format(row['id'])))

    fig.update_xaxes(title_text='step')
    fig.update_yaxes(title_text='accuracy')
    fig.update_layout(
        title=title)

    fig.show()

In [15]:
plot_accuracy_curve(df_gpt3, "Accuracy of the model in different versions of the dataset (augmentation by GPT-3).")

We observe that the accuracy of the model increases as we augment more and more data. After generating 200 extra synthetic samples, the accuracy exceeds 70% indicating that text augmentation can greatly improve the accuracy of our model.

In [16]:
plot_accuracy_curve(df_gptneo, "Accuracy of the model in different versions of the dataset (augmentation by GPT-Neo).")

The results of GPT-Neo are worse 😞. The performance of the model decreases when we generate synthetic data (except in the case of 50 synthetic samples).

In [17]:
plot_accuracy_curve(df_gptj, "Accuracy of the model in different versions of the dataset (augmentation by GPT-J).")

The performance curve of GPT-J is awesome 🎉. Each time we increase the size of the train dataset with augmentation, there is a consistent increase in the accuracy reaching 85%.

The next step is to investigate which language model boosts the final performance more.

In [19]:
# keep the accuracy of the last training step
acc_gpt3 = [df_baseline.iloc[0]['logs'][-1]['eval_accuracy']]
for i in range(4):
    acc_gpt3.append(df_gpt3.iloc[i]['logs'][-1]['eval_accuracy'])

acc_gptj = [df_baseline.iloc[0]['logs'][-1]['eval_accuracy']]
for i in range(4):
    acc_gptj.append(df_gptj.iloc[i]['logs'][-1]['eval_accuracy'])

acc_gptneo = [df_baseline.iloc[0]['logs'][-1]['eval_accuracy']]
for i in range(4):
    acc_gptneo.append(df_gptneo.iloc[i]['logs'][-1]['eval_accuracy'])

fig = go.Figure()

fig.add_trace(go.Scatter(
                x=[0, 10, 50, 100, 200],
                y=acc_gpt3,
                name='GPT-3'))


fig.add_trace(go.Scatter(
                x=[0, 10, 50, 100, 200],
                y=acc_gptj,
                name='GPT-J'))

fig.add_trace(go.Scatter(
                x=[0, 10, 50, 100, 200],
                y=acc_gptneo,
                name='GPT-Neo'))

fig.update_xaxes(title_text='number of synthetic samples')
fig.update_yaxes(title_text='accuracy')
fig.update_layout(
    title="Comparison of GPT-3, GPT-J and GPT-Neo in text augmentation.")

fig.show()

GPT-J outperforms GPT-3 in almost all different versions of the dataset 👏. These are very exciting results that indicate that we can use the open-source GPT-J instead of GPT-3 for augmentation. As for the GPT-Neo, it performs very poorly.

### Label Distribution

It is interesting to examine how the distribution of the labels changes when we generate more and more synthetic samples. Our initial train set is balanced since it consists of 10 samples per class. Let's see how the distribution of the labels changes after text augmentation! 

In [23]:
fig = make_subplots(rows=1, cols=4,
                    subplot_titles=("Baseline (30 samples)", "GPT-3 (200 synthetic samples)", "GPT-Neo (200 synthetic samples)", "GPT-J (200 synthetic samples)"))

trace0 = go.Histogram(x=[mapping.int2str(i) for i in emotion_train_ds["label"]],
                   opacity=0.8)

trace1 = go.Histogram(x=[mapping.int2str(i) for i in concatenate_datasets([emotion_train_ds, datasets_gpt3['200']])["label"]],
                   opacity=0.8)

trace2 = go.Histogram(x=[mapping.int2str(i) for i in concatenate_datasets([emotion_train_ds, datasets_gptneo['200']])["label"]],
                   opacity=0.8)

trace3 = go.Histogram(x=[mapping.int2str(i) for i in concatenate_datasets([emotion_train_ds, datasets_gptj['200']])["label"]],
                   opacity=0.8)

fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)
fig.append_trace(trace2, 1, 3)
fig.append_trace(trace2, 1, 4)
fig.update_layout(showlegend=False, title_text="Distribution of labels", 
                  bargap=0.30, width=1300)

fig.show()

We observe that the distribution changes a lot and the large augmented dataset is highly imbalanced 😯! GPT-3 model generated too many samples labeled as 'anger' while GPT-J and GPT-Neo generated more samples labeled as 'surprise'. The reason for this behavior is the datasets that these models are pre-trained on. GPT-J and GPT-Neo are trained on the same dataset and change the label distribution in a similar way.

That's an interesting observation that we should further examine in the future! 

## ⛏ Generate more samples

Text augmentation increased accuracy by a lot! The next step is to examine how much the accuracy improves when adding even more labelled samples. We expect that at some point, the accuracy curve stops increasing and stabilizes.


In [ ]:
# load the synthetic datasets with 300, 400 and 500 samples.
# run this if you do not want to generate new samples on your own.

if not os.path.isdir('./data'):
    !git clone https://github.com/ml6team/quick-tips.git
    !cd quick-tips && git checkout feature/nlp_gpt3mix
    !mv quick-tips/nlp/2021_11_25_gpt3mix/data ./data
    !rm -rf quick-tips

synthetic_gpt3_300_ds = load_from_disk('./data/gpt-3/300')
synthetic_gpt3_400_ds = load_from_disk('./data/gpt-3/400')
synthetic_gpt3_500_ds = load_from_disk('./data/gpt-3/500')

In [ ]:
# prepare datasets
max_size = 500
steps = 5*int(max_size/batch_size) # 4 epochs in the large dataset

datasets_gpt3_more = {'300': synthetic_gpt3_300_ds, '400': synthetic_gpt3_400_ds, '500': synthetic_gpt3_500_ds}

In case you loaded our results earlier, ignore the next cell that trains the model on the the new augmented datasets.

In [ ]:
# train the model on the augmented datasets of GPT-3
# in case you loaded our results earlier, ignore this cell

for i in datasets_gpt3_more:
    augmented_gpt3_ds = concatenate_datasets([emotion_train_ds, datasets_gpt3_more[i]])
    metrics, logs = train_and_evaluate(augmented_gpt3_ds, emotion_test_ds, "augmented_gpt3_" + i)

    run_dicts.append({
        "id": "augmented_gpt3_" + i,
        "metrics": metrics,
        "logs": logs
    })


In [24]:
df = pd.DataFrame(run_dicts)
gpt3_names_more = ['augmented_gpt3_10', 'augmented_gpt3_50', 'augmented_gpt3_100', 'augmented_gpt3_200', 
              'augmented_gpt3_300', 'augmented_gpt3_400', 'augmented_gpt3_500']

df_gpt3_more = df.loc[df['id'].isin(gpt3_names_more)]

In [26]:
acc_gpt3_more = [df_baseline.iloc[0]['logs'][-1]['eval_accuracy']]
for i in range(7):
    acc_gpt3_more.append(df_gpt3_more.iloc[i]['logs'][-1]['eval_accuracy'])

fig = go.Figure()

fig.add_trace(go.Scatter(
                x=[0, 10, 50, 100, 200, 300, 400, 500],
                y=acc_gpt3_more,
                name='GPT-3'))

fig.update_xaxes(title_text='number of extra samples')
fig.update_yaxes(title_text='accuracy')
fig.update_layout(showlegend=True, 
    title="Accuracy of the model in different versions of the dataset (augmentation by GPT-3).")

fig.show()

The curve goes up really fast in the beginning but the increase gradually slows down the more samples we generate. So, we realise that there is a limit in the performance improvements that text augmentation can yield.

## 🏁 Take-aways 


You've reached the finish line! 👏  Let's summarize some of the findings.

* We generated very realistic synthetic samples by leveraging the few-shot capabilities of large LMs and prompt engineering.
* As a baseline, we trained a distilbert model on the Emotion dataset using a small subset of 30 samples.
* Then, we augmented the small dataset with 10, 50, 100 and 200 extra samples generated by GPT-3, GPT-J and GPT-Neo.
* We compared the performance of the models in all these settings and showed that data augmentation boosts the performance 🥳.
* We showed that GPT-J performs better than GPT-3 in our task and can be used as an open-source alternative for text augmentation 💪. GPT-Neo performs poorly mainly because it is much smaller.
* However, the augmented datasets are not balanced anymore because large LMs are prone to generate samples with certain labels based on the data they are trained on.
* Finally, as we generate more and more synthetic samples and the size of the training set increases, the overall performance increases until some point. Then, text augmentation cannot improve the performance more and we should look into other ways of increasing performance.




